In [ ]:
# /notebooks/exploration.ipynb
[1] # Data Overview
import pandas as pd
from src.config import DATA_DIR

df = pd.concat([
    pd.read_csv(DATA_DIR / "raw/phishtank/samples.csv").assign(type="phishing"),
    pd.read_csv(DATA_DIR / "raw/enron/emails.csv").assign(type="legitimate")
])

[2] # Class Distribution
df["type"].value_counts().plot.pie(title="Dataset Balance")

[3] # Text Length Analysis
df["text_length"] = df["text"].str.len()
df.groupby("type")["text_length"].plot.hist(bins=50, alpha=0.5, legend=True)